Recall that the time-averaged drift velocity is the net displacement of the fluid element over a long period of time, divided by that time period which is approximately $X(T) / T$ for $X(0) = 0$. For sufficiently small $|a|$, we verify that the time-averaged mean drift velocity is $a^2/2$.

The fluid velocity is not static; it's a traveling wave $u = a \cos(2\pi(x - t))$ that propagates in the positive $x$-direction with a constant speed of $c=1$. The material element is advected by the fluid with velocity $dX/dt$ equal to the fluid's velocity at the element's current position $X(t)$. The maximum possible speed the fluid can impart to the element is given by the dimensionless amplitude $a$, which represents the ratio of the maximum fluid velocity to the wave propagation speed.

*   Sub-critical $|a| < 1$, trapped motion.
    
    The element is picked up by a region of positive velocity and moves forward. However, because the wave is faster, the element is overtaken by the region of negative velocity which pushes it backward. This leads to oscillatory motion but it also experiences a slow, net forward Stokes drift. This is the regime where the time-averaged drift velocity is $a^2/2$.

*   Super-critical $|a| > 1$, escaping motion.
    
    The fluid has enough strength to carry the element faster than the wave propagation. The element is untrapped and escapes the wave's oscillatory influence. The particle's velocity is always positive, even when the element passes through a weaker part of the velocity wave. The result is a continuous positive motion with only minor fluctuations in its speed.

*   Critical case $|a| = 1$, wave surfing.

    The particle is perfectly captured by the wave. Its velocity remains constant and positive, and it moves along with the wave crest without oscillating.

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pandas as pd

def fluid_trajectory_ode(t, X, a):
    '''
    Defines the ordinary differential equation for the fluid element's trajectory.
    '''
    return a * np.cos(2 * np.pi * (X - t))

def verify_drift_velocity():
    '''
    Calculates and verifies the time-averaged drift velocity for small 'a'.
    '''
    small_a_values = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5]
    t_final = 200.0
    t_span = [0, t_final]
    X0 = [0]
    results = []

    for a_val in small_a_values:
        solution = solve_ivp(
            fluid_trajectory_ode,
            t_span,
            X0,
            args=(a_val,),
            rtol=1e-8,  # Use a tight tolerance for accuracy
            atol=1e-8
        )

        final_position = solution.y[0][-1]
        numerical_drift = final_position / t_final
        theoretical_drift = 0.5 * a_val**2
        error = np.abs((numerical_drift - theoretical_drift) / theoretical_drift) * 100
        results.append({
            "a": a_val,
            "Numerical Drift": numerical_drift,
            "Theoretical Drift (0.5*a^2)": theoretical_drift,
            "% Error": error
        })

    results_df = pd.DataFrame(results)
    print(results_df.to_string(index=False))

verify_drift_velocity()

   a  Numerical Drift  Theoretical Drift (0.5*a^2)  % Error
0.01         0.000050                      0.00005 0.990233
0.10         0.005011                      0.00500 0.227496
0.20         0.020167                      0.02000 0.834792
0.30         0.045833                      0.04500 1.851489
0.40         0.083817                      0.08000 4.771150
0.50         0.134349                      0.12500 7.479043


Define a new variable that represents the position of the material element relative to the moving wave's frame of reference $\phi(t) = X(t) - t$. Differentiating with respect to time $t$, we get
\begin{equation}
    \frac{d\phi}{dt} = \frac{dX}{dt} - 1.
\end{equation}
We then substitute the original equation $\frac{dX}{dt} = a \cos 2\pi(X - t)$, into this expression, which yields
\begin{equation}
    \frac{d\phi}{dt} = a \cos(2\pi\phi) - 1.
We have thus transformed the original ODE into an autonomous ODE.

The long-term behavior of the system can be understood by finding the fixed points of the transformed equation. Setting $\frac{d\phi}{dt} = 0$, we have
\begin{equation}
    a \cos(2\pi\phi) - 1 = 0 \implies \cos(2\pi\phi) = \frac{1}{a}.
\end{equation}

*   If $|a| > 1$, then $|1/a| < 1$ and there are real solutions for $\phi$. This means fixed points exist. A particle that reaches a fixed point $\phi^*$ will maintain a constant position relative to the moving wave. Its trajectory will be $X(t) = t + \phi^*$. The particle's long-term velocity $dX/dt$ becomes exactly $1$, which is the speed of the wave. Analysis shows that for each integer $n$, there is one stable and one unstable fixed point, meaning particles are naturally drawn into the surfing state.

*   If $|a| < 1$, then $|1/a| > 1$ and there is no real value of $\phi$ that can satisfy $\cos(2\pi\phi) = 1/a$. Therefore, no fixed points exist.
Since $d\phi/dt$ can never be zero, by continuity, it always be negative as the term $a \cos(2\pi\phi) - 1$ is always negative. This means $\phi(t) = X(t) - t$ is a strictly decreasing function. The material element is always slipping back relative to the moving wave frame, causing the oscillatory motion.

We expresse the solution $X(t)$ as a power series in the small parameter $a$, with $X(0) = 0$,
\begin{equation}
    X(t) = X_0(t) + a X_1(t) + a^2 X_2(t) + O(a^3).
\end{equation}
Substituting this into the ODE $dX/dt = a\cos(2\pi(X - t))$ and performing a Taylor expansion of the cosine term gives a series equations for each order of $a$.
*   Zeroth order: $dX_0/dt = 0 = X_0(t)$. This states that with no flow (a=0), the particle doesn't move.
*   First order: $dX_1/dt = \cos(2\pi(X_0 - t)) = \cos(-2\pi t) = \cos(2\pi t)$. Integrating this gives the first-order motion $X_1(t) = \frac{1}{2\pi} \sin(2\pi t)$. This is a purely oscillatory motion with zero average velocity.
*   Second order: $dX_2/dt = -2\pi X_1(t) \sin(2\pi(X_0 - t))$. Substituting the solutions for $X_0$ and $X_1$ yields
\begin{equation}
    \frac{dX_2}{dt} = -2\pi \left[\frac{1}{2\pi}\sin(2\pi t)\right] \sin(-2\pi t) = \sin^2(2\pi t) = \frac{1}{2}(1-\cos(4\pi t)).
\end{equation}
The mean velocity is the time-average of the velocity expression over a long period,
\begin{equation}
    \left\langle\frac{dX}{dt}\right\rangle = \left\langle a \frac{dX_1}{dt} + a^2 \frac{dX_2}{dt} + \cdots\right\rangle = a \langle\cos(2\pi t)\rangle + \frac{1}{2}a^2 \langle (1 - \cos(4\pi t))\rangle + O(a^3)
\end{equation}
The time-average of the cosine functions over many cycles is zero. Thus, we are left with $\langle dX/dt \rangle \approx a^2/2$.This shows that the net drift is a second-order effect arising from the non-linear interaction between the particle's induced oscillation and the velocity field.